### Réentrainement à partir d'un run MLFLOW
L'objectif de ce notebook est de voir comment :
1. récupérer un run mlflow
2. retourner au commit dvc correspondant à ce run
3. relancer l'entrainement

In [1]:
import dagshub
import mlflow
from mlflow.tracking import MlflowClient
import subprocess
import json
dagshub.init(repo_owner='Belwen', repo_name='SEP24-MLOPS-SAKE', mlflow=True)

Accessing as Belwen

Initialized MLflow to track repo "Belwen/SEP24-MLOPS-SAKE"

Repository Belwen/SEP24-MLOPS-SAKE initialized!

In [7]:

import os
from dvc.repo import Repo
from dvc.exceptions import DvcException

def get_dvc_commit_hash():
    """
    Get the current DVC commit hash from the repository.
    
    Returns:
        str: The current DVC commit hash
        
    Raises:
        DvcException: If there's an error accessing the DVC repository
        ValueError: If not in a DVC repository
    """
    try:
        # Initialize DVC repo object
        repo = Repo(".")
        
        # Get the current commit hash
        commit_hash = repo.scm.get_rev()
        
        return commit_hash
        
    except DvcException as e:
        raise DvcException(f"Error accessing DVC repository: {str(e)}")
    except Exception as e:
        raise ValueError(f"Not in a DVC repository or other error occurred: {str(e)}")

def get_dvc_commit_from_mlflow_run(run_id):
    """
    Récupère le commit DVC associé à un run MLflow spécifique.
    
    Args:
        run_id (str): L'ID du run MLflow
    
    Returns:
        str: Le hash du commit DVC
    """
    client = MlflowClient()
    run = client.get_run(run_id)
    
    # Récupérer le hash DVC des tags MLflow
    dvc_commit = run.data.tags.get('dvc_commit')
    if not dvc_commit:
        raise ValueError("Aucun commit DVC trouvé pour ce run MLflow")
    
    return dvc_commit

def checkout_dvc_commit(dvc_commit):
    """
    Checkout l'état DVC spécifique.
    
    Args:
        dvc_commit (str): Le hash du commit DVC
    """
    # Checkout du commit DVC
    subprocess.run(['dvc', 'checkout', dvc_commit], check=True)

def reproduce_mlflow_experiment(run_id):
    """
    Reproduit un experiment MLflow en restaurant l'état DVC correspondant.
    
    Args:
        run_id (str): L'ID du run MLflow à reproduire
    """
    # 1. Récupérer le commit DVC
    dvc_commit = get_dvc_commit_from_mlflow_run(run_id)
    
    # 2. Restaurer l'état DVC
    checkout_dvc_commit(dvc_commit)
    
    # 3. Récupérer les paramètres du run original
    client = MlflowClient()
    original_run = client.get_run(run_id)
    params = original_run.data.params
    
    # 4. Relancer l'expérience à partir du preprocessing
    subprocess.run(['dvc', 'repro preprocessing'], check=True)



In [11]:
get_dvc_commit_hash()

'0a6823f6bd16cb89c39a675b16f5fdc490c7c65e'